<a href="https://colab.research.google.com/github/syh0397/Statistics_python/blob/main/7_Anova_test(Two_way).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pingouin
!pip install --upgrade pingouin
!pip install scikit_posthocs

## 이원배치분산분석(Two-Way ANOVA)(Analysis of Variance)

#### 설명

- 일원배치 = 독립변수가 하나 
- 이원배치 =  독립변수 두개 

    - 두개의 범주형 독립변수가  하나의 연속형 종속변수에 미치는 영향을 본다 .   평균차이를 본다.  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd 
import numpy as np 

import os
import sys
import warnings
warnings.filterwarnings('ignore')

import sklearn 
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=2.5)

import scipy 
from scipy import  stats
import statsmodels.api as sm
import pylab as py

import pingouin as pg
import scikit_posthocs

In [4]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'geyser',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'taxis',
 'tips',
 'titanic']

In [5]:
a = sns.load_dataset('tips')
a

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


- 이원배치분산분석에서는 두개의 상호작용도 볼 수 있다
- 아래에서 day size 두개의 상호작용에 대한 효과가 total_bill 의 평균 차이에 나타나는가? 

In [6]:
hos = a[['day','size']]
tip = a['tip']
hos['tip'] = tip
hos

,day,size,tip
0,Sun,2,1.01
1,Sun,3,1.66
2,Sun,3,3.50
3,Sun,2,3.31
4,Sun,4,3.61
...,...,...,...
239,Sat,3,5.92
240,Sat,2,2.00
241,Sat,2,2.00
242,Sat,2,1.75


In [7]:
print(pg.anova(dv = 'tip',
               between = ['day', 'size'],
               data = hos))

       Source          SS     DF          MS           F         p-unc  \
0         day   26.436757    3.0    8.812252    5.861675  7.180854e-04   
1        size  995.012636    5.0  199.002527  132.371166  7.808818e-65   
2  day * size  111.289850   15.0    7.419323    4.935136  2.512425e-08   
3    Residual  336.754351  224.0    1.503368         NaN           NaN   

        np2  
0  0.072790  
1  0.747137  
2  0.248390  
3       NaN  
